In [13]:
from sklearn.preprocessing import StandardScaler
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [14]:
import mysql.connector
import pandas as pd

import json

with open('config.json') as f:
    config = json.load(f)


cnx = mysql.connector.connect(user=config['user'],
                              password=config['password'],
                              host=config['host'],
                              database=config['database'])

cursor = cnx.cursor()

# Example query
query = ("SELECT * FROM PH_Temp_Diet")

cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()

# Create a pandas DataFrame
data = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])


cursor.close()
cnx.close()

In [15]:
df3 =data
df3 = df3.drop(['id'], axis=1)

# Pivot the data to create columns for each unique food_name
pivoted_data = df3.pivot_table(index=['person_id', 'date','PH','temperature'], columns='food_name', values='amount')
pivoted_data = pivoted_data.drop('person_id', axis=1)
pivoted_data = pivoted_data.reset_index()
# Rename columns to remove the 'food_name' label prefix
pivoted_data.columns.name = None
pivoted_data.columns = pivoted_data.columns.str.replace('food_name_', '')

# # Join the pivoted data with the original data on the 'person_id' and 'date' columns
joined_data = pd.merge(data.drop(columns=['food_name', 'amount','id']), pivoted_data, on=['person_id', 'date','temperature','PH'])

# # Save the joined data to a new CSV file
#df3 = pivoted_data
# Group by 'person_id' and 'date', and aggregate the other columns
grouped_df = joined_data.groupby(['person_id', 'date','temperature','PH'], as_index=False);
joined_data = joined_data.drop_duplicates()
joined_data

,PH,temperature,date,person_id,Apple,Apple Pie,Banana,Beer,Braised Potatoes,Bread,...,Tangerines,Tea(black/green),Tequila,The nuts,The vinaigrette,Vegetable salad,Water,Watermelon,White wine,Zucchini Fritters
0,5.40,35.8,2017-09-06,1,0.0,0.0,100.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,150.0
97,7.50,36.1,2017-09-07,1,0.0,0.0,0.0,0.0,0.0,180.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,50.0
190,5.81,36.4,2017-09-08,1,0.0,0.0,0.0,0.0,0.0,250.0,...,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,50.0
283,7.50,36.6,2017-09-09,1,0.0,0.0,110.0,500.0,0.0,100.0,...,0.0,0.0,0.0,60.0,0.0,0.0,400.0,0.0,0.0,0.0
376,7.50,35.9,2017-09-10,1,0.0,0.0,100.0,0.0,0.0,150.0,...,0.0,0.0,0.0,30.0,0.0,0.0,300.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19720,5.40,35.8,2017-09-06,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19721,5.40,35.8,2023-04-16,1,NaN,0.0,0.0,500.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19905,5.40,35.8,2023-04-20,1,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20181,5.40,35.8,2023-04-22,1,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Define a function to recommend food items based on pH level
def recommend_foods(ph_level, df):
    # Calculate the mean pH for each food item
    food_phs = df.iloc[:, 5:].mean()
    # Determine which food items are associated with low or high pH levels
    low_phs = food_phs[food_phs < ph_level].index.tolist()
    high_phs = food_phs[food_phs > ph_level].index.tolist()
    
    # Return the recommended food items
    if ph_level < 7:
        return high_phs
    else:
        return low_phs

In [17]:
recommend_foods(6.8,joined_data)

['Banana',
 'Beer',
 'Braised Potatoes',
 'Bread',
 'Cake',
 'Cheese',
 'Cookie',
 'Croissant',
 'Curd',
 'Curd Pie',
 'Dried apricots',
 'Fried potatoes',
 'Fruit tea',
 'Ice cream',
 'Latte',
 'Lemon water',
 'Liquid',
 'Mashed potatoes',
 'Mayonnaise salad',
 'Meat',
 'Meat pilaf',
 'Meatless pilaf',
 'Melon',
 'Mineral water (Borjomi)',
 'Mineral water (Esentuki - 4)',
 'Mineral water (Prolom)',
 'Morse',
 'Noodles with shrimp',
 'Oatmeal',
 'Orange juice',
 'Patty with cabbage',
 'Peach',
 'Pear',
 'Pizza with meat',
 'Plum',
 'Potato dumplings',
 'Pumpkin seeds',
 'Red wine',
 'Scrambled eggs',
 'Sushi',
 'Tangerines',
 'Tea(black/green)',
 'The nuts',
 'Vegetable salad',
 'Water',
 'Watermelon',
 'White wine',
 'Zucchini Fritters']

In [18]:
df2= data
df2.head(5)

,id,PH,temperature,food_name,amount,date,person_id
0,39446,5.4,35.8,Gym,0.0,2017-09-06,1
1,39447,5.4,35.8,Liquid,1300.0,2017-09-06,1
2,39448,5.4,35.8,Lemon water,250.0,2017-09-06,1
3,39449,5.4,35.8,Tea(black/green),0.0,2017-09-06,1
4,39450,5.4,35.8,Fruit tea,600.0,2017-09-06,1


In [19]:
from sklearn.model_selection import train_test_split

X= joined_data.iloc[:,0:1]
X

,PH
0,5.40
97,7.50
190,5.81
283,7.50
376,7.50
...,...
19720,5.40
19721,5.40
19905,5.40
20181,5.40


In [20]:
y = joined_data.iloc[:,5:]
y

,Apple Pie,Banana,Beer,Braised Potatoes,Bread,Buckwheat porridge,Bun,Cake,Candies,Cheese,...,Tangerines,Tea(black/green),Tequila,The nuts,The vinaigrette,Vegetable salad,Water,Watermelon,White wine,Zucchini Fritters
0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,150.0
97,0.0,0.0,0.0,0.0,180.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,50.0
190,0.0,0.0,0.0,0.0,250.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,50.0
283,0.0,110.0,500.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,60.0,0.0,0.0,400.0,0.0,0.0,0.0
376,0.0,100.0,0.0,0.0,150.0,0.0,150.0,0.0,0.0,0.0,...,0.0,0.0,0.0,30.0,0.0,0.0,300.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19721,0.0,0.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
y = data.groupby(['person_id', 'date'])[['food_name', 'amount']].agg(list)
# Reset the index
y = y.reset_index()

# Custom function to combine food_name and amount into one list
def combine_food_and_amount(food_names, amounts):
    combined = []
    for food, amount in zip(food_names, amounts):
        combined.append((food, amount))
    return combined

# Apply the custom function to combine food_name and amount columns
y['food_and_amount'] = y.apply(lambda row: combine_food_and_amount(row['food_name'], row['amount']), axis=1)

# Drop the food_name and amount columns
y = y.drop(['food_name', 'amount','person_id','date'], axis=1)

# Print the modified DataFrame
print(y.head(5))

                                     food_and_amount
0  [(Gym, 0.0), (Liquid, 1300.0), (Lemon water, 2...
1  [(Gym, 1.0), (Liquid, 1300.0), (Lemon water, 2...
2  [(Gym, 0.0), (Liquid, 1350.0), (Lemon water, 2...
3  [(Gym, 0.0), (Liquid, 2350.0), (Lemon water, 2...
4  [(Gym, 1.0), (Liquid, 1600.0), (Lemon water, 2...


In [22]:
# import pandas as pd

# # Sample input data
# input_data = pd.DataFrame(y)

# # Function to process a single row
# def process_row(row):
#     return ', '.join([f'{food}: {amount}' for food, amount in row['food_and_amount']])

# # Process each row and store the result in a new column
# input_data['food_and_amount_str'] = input_data.apply(process_row, axis=1)

# # Drop the original food_and_amount column
# y = input_data.drop(['food_and_amount'], axis=1)

# print(y)

In [23]:
y = data.groupby(['person_id', 'date'])[['food_name', 'amount']].agg(list)
# Reset the index
y = y.reset_index()

# Custom function to combine food_name and amount into one list
def combine_food_and_amount(food_names, amounts):
    combined = []
    for food, amount in zip(food_names, amounts):
        combined.append((food, amount))
    return combined

# Apply the custom function to combine food_name and amount columns
y['food_and_amount'] = y.apply(lambda row: combine_food_and_amount(row['food_name'], row['amount']), axis=1)

# Drop the food_name and amount columns
y = y.drop(['food_name', 'amount','person_id','date'], axis=1)

# Print the modified DataFrame
print(y.head(5))

                                     food_and_amount
0  [(Gym, 0.0), (Liquid, 1300.0), (Lemon water, 2...
1  [(Gym, 1.0), (Liquid, 1300.0), (Lemon water, 2...
2  [(Gym, 0.0), (Liquid, 1350.0), (Lemon water, 2...
3  [(Gym, 0.0), (Liquid, 2350.0), (Lemon water, 2...
4  [(Gym, 1.0), (Liquid, 1600.0), (Lemon water, 2...


In [24]:
X

,PH
0,5.40
97,7.50
190,5.81
283,7.50
376,7.50
...,...
19720,5.40
19721,5.40
19905,5.40
20181,5.40


In [25]:
y

,food_and_amount
0,"[(Gym, 0.0), (Liquid, 1300.0), (Lemon water, 2..."
1,"[(Gym, 1.0), (Liquid, 1300.0), (Lemon water, 2..."
2,"[(Gym, 0.0), (Liquid, 1350.0), (Lemon water, 2..."
3,"[(Gym, 0.0), (Liquid, 2350.0), (Lemon water, 2..."
4,"[(Gym, 1.0), (Liquid, 1600.0), (Lemon water, 2..."
...,...
212,"[(Gym, 0.0), (Liquid, 0.0), (Lemon water, 0.0)..."
213,"[(Gym, 100.0), (Liquid, 0.0), (Lemon water, 0...."
214,"[(Gym, 0.0), (Liquid, 0.0), (Lemon water, 0.0)..."
215,"[(Gym, 0.0)]"


In [26]:
df = pd.DataFrame(y)
df2 = pd.DataFrame(X)
# convert column 'A' to a tuple
food_and_amount = tuple(df['food_and_amount'].tolist())
ph = tuple(df2['PH'].tolist())
print(ph)

(5.4, 7.5, 5.81, 7.5, 7.5, 5.67, 7.5, 5.81, 6.0, 7.5, 7.5, 5.94, 7.5, 7.5, 7.5, 7.5, 7.5, 5.48, 7.5, 5.6, 7.5, 7.5, 7.5, 5.87, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 5.97, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 5.99, 5.63, 5.65, 7.5, 5.97, 4.98, 5.14, 7.5, 7.5, 5.35, 7.5, 5.86, 5.69, 5.31, 5.43, 5.12, 5.85, 5.95, 7.5, 5.96, 5.64, 7.5, 5.68, 5.73, 7.5, 5.97, 5.94, 5.99, 7.5, 5.52, 5.68, 7.5, 5.88, 7.5, 5.47, 5.72, 4.96, 5.67, 7.5, 7.5, 7.5, 5.95, 7.5, 7.5, 5.95, 7.5, 5.51, 5.38, 5.34, 5.71, 7.5, 5.57, 5.02, 5.47, 5.24, 7.5, 5.57, 7.5, 5.79, 5.5, 7.5, 5.99, 5.56, 5.38, 5.59, 5.46, 7.5, 5.07, 5.77, 5.87, 7.5, 7.5, 5.54, 5.89, 5.6, 5.97, 7.5, 7.5, 7.5, 7.5, 7.5, 5.77, 7.5, 7.5, 7.5, 7.5, 5.99, 7.5, 5.98, 7.5, 5.81, 7.5, 7.5, 7.5, 7.5, 7.5, 5.81, 5.77, 7.5, 7.5, 7.5, 6.0, 5.86, 7.5, 5.89, 7.5, 5.84, 7.5, 7.5, 5.74, 5.69, 5.35, 5.59, 5.77, 5.54, 5.39, 5.33, 5.52, 7.5

In [27]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Replace this with your actual dataset
# data = pd.DataFrame({
#     'pH_level': [5.0, 6.0, 7.0, 5.5],
#     'food_and_activities': [
#         [('Gym', 0.0), ('Liquid', 2920.0), ('Lemon water', 300.0)],
#         [('Gym', 1.0), ('Liquid', 2000.0), ('Lemon water', 100.0)],
#         [('Gym', 0.0), ('Liquid', 2500.0), ('Lemon water', 0.0)],
#         [('Gym', 1.0), ('Liquid', 1800.0), ('Lemon water', 200.0)],
#     ]
# })

data = pd.DataFrame({
    'pH_level': ph,
    'food_and_activities': food_and_amount
})

def categorize_pH(pH):
    if pH < 7.0:
        return "acidic"
    elif pH == 7.0:
        return "neutral"
    else:
        return "alkaline"

data['pH_category'] = data['pH_level'].apply(categorize_pH)

def binary_representation(row):
    binary_list = [row['pH_category']]
    for food, amount in row['food_and_activities']:
        if amount > 0:
            binary_list.append(food)
    return binary_list

data['binary'] = data.apply(binary_representation, axis=1)

transactions = data['binary'].tolist()

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
binary_df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(binary_df, min_support=0.5, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

# Filter the rules based on the antecedent (left-hand side) containing the pH category "acidic"
acidic_rules = rules[rules['antecedents'].apply(lambda x: 'acidic' in x)]

print(acidic_rules)


   antecedents consequents  antecedent support  consequent support   support  \
68    (acidic)    (Liquid)            0.520737            0.981567  0.502304   

    confidence      lift  leverage  conviction  
68    0.964602  0.982716 -0.008834    0.520737  


In [28]:
def recommend_food_and_activities(pH_level, rules):
    pH_category = categorize_pH(pH_level)

    # Filter the rules based on the antecedent (left-hand side) containing the pH category
    recommendations = rules[rules['antecedents'].apply(lambda x: pH_category in x)]

    # Sort the rules by confidence and select the top 5 rules
    top_recommendations = recommendations.sort_values(by='confidence', ascending=False).head(5)

    # Extract the consequents (right-hand side) of the rules
    recommended_items = set()
    for _, row in top_recommendations.iterrows():
        recommended_items.update(row['consequents'])

    # Remove the pH category from the recommended items
    recommended_items.discard(pH_category)

    return list(recommended_items)


# Example usage
urine_pH = 2# Replace this with the actual pH level data from the sensor
recommended_items = recommend_food_and_activities(urine_pH, acidic_rules)
print("Recommended food and activities:", recommended_items)


Recommended food and activities: ['Liquid']


In [29]:
import joblib

# Save the association rules DataFrame to a file
joblib.dump(acidic_rules, 'acidic_rules.joblib')

# Save the categorize_pH function to a file
joblib.dump(categorize_pH, 'categorize_pH_function.joblib')

['categorize_pH_function.joblib']

In [30]:
import joblib

# Load the association rules DataFrame and the categorize_pH function
acidic_rules = joblib.load('acidic_rules.joblib')
categorize_pH = joblib.load('categorize_pH_function.joblib')

# Use the loaded model and function to make recommendations
urine_pH = 5.5  # Replace this with the actual pH level data from the sensor
recommended_items = recommend_food_and_activities(urine_pH, acidic_rules)
print("Recommended food and activities:", recommended_items)


Recommended food and activities: ['Liquid']
